In [15]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
df = pd.read_csv('resources/original_dataset.csv', low_memory=False)
df

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,RSBP,RCONSC,RDEF1,...,DEAD8,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14
0,M,69,Y,NaN,Y,Y,NaN,140,D,N,...,0,0,0,0,0,0,0,0,0,0
1,M,76,Y,NaN,Y,N,NaN,150,F,Y,...,0,0,0,0,0,0,0,0,0,0
2,F,71,N,NaN,Y,N,NaN,170,F,Y,...,0,0,0,0,0,0,0,0,0,0
3,M,81,N,NaN,N,N,NaN,170,F,N,...,0,0,0,0,0,0,0,0,0,0
4,M,78,N,NaN,N,N,NaN,170,F,Y,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19430,M,66,N,N,Y,Y,N,120,F,Y,...,0,0,0,0,0,0,0,0,0,1
19431,M,75,N,N,Y,Y,N,210,F,Y,...,0,0,0,0,0,0,0,0,0,0
19432,M,77,N,N,Y,N,N,220,F,Y,...,0,0,1,0,1,0,0,0,0,0
19433,F,87,N,N,Y,Y,N,160,D,N,...,0,0,0,0,0,0,0,0,0,0


In [16]:
print(df.dtypes)


SEX        object
AGE         int64
RSLEEP     object
RATRIAL    object
RCT        object
            ...  
HTI14       int64
PE14        int64
DVT14       int64
TRAN14      int64
NCB14       int64
Length: 77, dtype: object


In [17]:
object_columns = df.select_dtypes(include='object').columns
print(object_columns)


Index(['SEX', 'RSLEEP', 'RATRIAL', 'RCT', 'RVISINF', 'RHEP24', 'RCONSC',
       'RDEF1', 'RDEF2', 'RDEF3', 'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8',
       'STYPE', 'RDATE', 'RXASP', 'RXHEP', 'DASP14', 'DASPLT', 'DLH14',
       'DMH14', 'DHH14', 'DSCH', 'DIVH', 'DAP', 'DOAC', 'DGORM', 'DSTER',
       'DCAA', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA',
       'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH', 'DRSUNK', 'DPE', 'DALIVE',
       'DPLACE', 'DDEAD', 'FPLACE', 'FAP', 'FOAC', 'COUNTRY'],
      dtype='object')


In [18]:
df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
df.head(10)

,SEX,AGE,RSLEEP,RATRIAL,RCT,RVISINF,RHEP24,RSBP,RCONSC,RDEF1,...,DEAD8,H14,ISC14,NK14,STRK14,HTI14,PE14,DVT14,TRAN14,NCB14
0,M,69,Y,NaN,Y,Y,NaN,140,D,N,...,0,0,0,0,0,0,0,0,0,0
1,M,76,Y,NaN,Y,N,NaN,150,F,Y,...,0,0,0,0,0,0,0,0,0,0
2,F,71,N,NaN,Y,N,NaN,170,F,Y,...,0,0,0,0,0,0,0,0,0,0
3,M,81,N,NaN,N,N,NaN,170,F,N,...,0,0,0,0,0,0,0,0,0,0
4,M,78,N,NaN,N,N,NaN,170,F,Y,...,0,0,0,0,0,0,0,0,0,0
5,M,54,N,NaN,Y,N,NaN,135,F,Y,...,0,0,0,0,0,0,0,0,0,0
6,F,77,N,NaN,N,N,NaN,140,F,Y,...,0,0,0,0,0,0,0,0,0,1
7,M,23,N,NaN,Y,N,NaN,120,F,N,...,0,0,0,0,0,0,0,0,0,0
8,M,47,N,NaN,N,N,NaN,150,F,N,...,0,0,0,0,0,0,0,0,0,0
9,M,81,Y,NaN,N,N,NaN,170,F,N,...,0,0,0,0,0,0,0,0,0,0


In [19]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)


RATRIAL       984
RHEP24        344
DASP14         22
DASPLT        147
DLH14          22
DMH14        1006
DHH14       18451
ONDRUG          1
DSCH          305
DIVH          305
DAP            18
DOAC           18
DGORM          23
DSTER          28
DCAA           29
DCAREND      1005
DTHROMB       315
DMAJNCH        14
DDIAGISC       23
DDIAGHA        26
DDIAGUN        23
DNOSTRK        26
DRSISC         18
DRSH           15
DRSUNK       1007
DPE            14
DALIVE         28
DPLACE       9709
DDEAD          19
DDEADC      17101
FPLACE       4502
FAP          4568
FOAC         4574
dtype: int64


In [20]:
C_columns = ['RATRIAL', 'RHEP24']
for col in C_columns:
    df[col] = df[col].fillna('C')


In [21]:
U_columns = ['DASP14', 'DASPLT', 'DLH14', 'DMH14', 'DHH14', 'DSCH', 'DIVH', 'DAP', 'DOAC', 'DGORM', 'DSTER',
             'DCAA', 'DCAREND', 'DTHROMB', 'DMAJNCH', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'DRSISC', 'DRSH',
             'DRSUNK', 'DPE', 'DALIVE', 'DPLACE', 'DDEAD', 'FPLACE', 'FAP', 'FOAC']
for col in U_columns:
    df[col] = df[col].fillna('U')


In [22]:
numeric_columns = ['DDEADC', 'ONDRUG']
for col in numeric_columns:
    df[col] = df[col].fillna(0)


In [23]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)


Series([], dtype: int64)


In [24]:
for col in object_columns:
    print(f"Column: {col}")
    print(df[col].unique())


Column: SEX
['M' 'F']
Column: RSLEEP
['Y' 'N']
Column: RATRIAL
['C' 'Y' 'N']
Column: RCT
['Y' 'N']
Column: RVISINF
['Y' 'N']
Column: RHEP24
['C' 'N' 'Y']
Column: RCONSC
['D' 'F' 'U']
Column: RDEF1
['N' 'Y' 'C']
Column: RDEF2
['N' 'Y' 'C']
Column: RDEF3
['N' 'Y' 'C']
Column: RDEF4
['Y' 'N' 'C']
Column: RDEF5
['N' 'Y' 'C']
Column: RDEF6
['Y' 'N' 'C']
Column: RDEF7
['N' 'Y' 'C']
Column: RDEF8
['Y' 'N' 'C']
Column: STYPE
['PACS' 'LACS' 'POCS' 'OTH' 'TACS']
Column: RDATE
['1991-01' '1991-02' '1991-03' '1991-04' '1991-05' '1991-06' '1991-07'
 '1991-08' '1991-09' '1991-10' '1991-11' '1991-12' '1992-01' '1992-02'
 '1992-03' '1992-04' '1992-05' '1992-06' '1992-07' '1992-08' '1992-09'
 '1992-10' '1992-11' '1992-12' '1993-01' '1993-02' '1993-03' '1993-07'
 '1994-01' '1993-09' '1993-08' '1993-12' '1993-11' '1993-04' '1994-03'
 '1994-09' '1994-07' '1994-08' '1993-05' '1993-10' '1994-04' '1994-02'
 '1994-05' '1993-06' '1994-06' '1994-12' '1995-01' '1994-10' '1994-11'
 '1995-02' '1995-07' '1996-05' '

In [25]:
for col in object_columns:
    print(f"Column: {col}")
    print(df[col].value_counts())


Column: SEX
SEX
M    10407
F     9028
Name: count, dtype: int64
Column: RSLEEP
RSLEEP
N    13750
Y     5685
Name: count, dtype: int64
Column: RATRIAL
RATRIAL
N    15282
Y     3169
C      984
Name: count, dtype: int64
Column: RCT
RCT
Y    13024
N     6411
Name: count, dtype: int64
Column: RVISINF
RVISINF
N    13020
Y     6415
Name: count, dtype: int64
Column: RHEP24
RHEP24
N    18655
Y      436
C      344
Name: count, dtype: int64
Column: RCONSC
RCONSC
F    14921
D     4254
U      260
Name: count, dtype: int64
Column: RDEF1
RDEF1
Y    14099
N     5089
C      247
Name: count, dtype: int64
Column: RDEF2
RDEF2
Y    16645
N     2667
C      123
Name: count, dtype: int64
Column: RDEF3
RDEF3
Y    14678
N     4502
C      255
Name: count, dtype: int64
Column: RDEF4
RDEF4
N    10342
Y     8509
C      584
Name: count, dtype: int64
Column: RDEF5
RDEF5
N    12394
C     3945
Y     3096
Name: count, dtype: int64
Column: RDEF6
RDEF6
N    12809
C     3448
Y     3178
Name: count, dtype: int64
Column: RDE

In [26]:
# Define ternary mapping
ternary_map = {'Y': 1, 'N': 0, 'U': 2, 'C': 2}

# Define columns to exclude
excluded_columns = ['SEX', 'RCONSC', 'STYPE', 'RDATE', 'COUNTRY', 'RXHEP', 'DPLACE', 'FPLACE']

# Apply ternary encoding to columns
for col in object_columns:
    if col not in excluded_columns:
        df[col] = df[col].map(ternary_map)

# Check
print(df.head())


  SEX  AGE  RSLEEP  RATRIAL  RCT  RVISINF  RHEP24  RSBP RCONSC  RDEF1  ...  \
0   M   69       1        2    1        1       2   140      D      0  ...   
1   M   76       1        2    1        0       2   150      F      1  ...   
2   F   71       0        2    1        0       2   170      F      1  ...   
3   M   81       0        2    0        0       2   170      F      0  ...   
4   M   78       0        2    0        0       2   170      F      1  ...   

   DEAD8  H14  ISC14  NK14  STRK14  HTI14  PE14 DVT14 TRAN14  NCB14  
0      0    0      0     0       0      0     0     0      0      0  
1      0    0      0     0       0      0     0     0      0      0  
2      0    0      0     0       0      0     0     0      0      0  
3      0    0      0     0       0      0     0     0      0      0  
4      0    0      0     0       0      0     0     0      0      0  

[5 rows x 77 columns]


In [27]:
for col in object_columns:
    print(f"Column: {col}")
    print(df[col].isnull().sum())  # Count unexpected values


Column: SEX
0
Column: RSLEEP
0
Column: RATRIAL
0
Column: RCT
0
Column: RVISINF
0
Column: RHEP24
0
Column: RCONSC
0
Column: RDEF1
0
Column: RDEF2
0
Column: RDEF3
0
Column: RDEF4
0
Column: RDEF5
0
Column: RDEF6
0
Column: RDEF7
0
Column: RDEF8
0
Column: STYPE
0
Column: RDATE
0
Column: RXASP
0
Column: RXHEP
0
Column: DASP14
0
Column: DASPLT
0
Column: DLH14
0
Column: DMH14
0
Column: DHH14
0
Column: DSCH
0
Column: DIVH
0
Column: DAP
0
Column: DOAC
0
Column: DGORM
0
Column: DSTER
0
Column: DCAA
0
Column: DCAREND
0
Column: DTHROMB
0
Column: DMAJNCH
0
Column: DDIAGISC
0
Column: DDIAGHA
0
Column: DDIAGUN
0
Column: DNOSTRK
0
Column: DRSISC
0
Column: DRSH
0
Column: DRSUNK
0
Column: DPE
0
Column: DALIVE
0
Column: DPLACE
0
Column: DDEAD
0
Column: FPLACE
0
Column: FAP
0
Column: FOAC
0
Column: COUNTRY
0


In [28]:
# RDate in Date-Time format
df['RDATE'] = pd.to_datetime(df['RDATE'], format='%Y-%m')

# Checking
print(df['RDATE'].head())

0   1991-01-01
1   1991-01-01
2   1991-01-01
3   1991-01-01
4   1991-02-01
Name: RDATE, dtype: datetime64[ns]


In [29]:
# Save the updated DataFrame to a CSV file
df.to_csv('cleaned_data_12-9.csv', index=False)

print("File saved as 'cleaned_data_12-9.csv'.")

File saved as 'cleaned_data_12-9.csv'.
